In [2]:
import pandas as pd
import numpy as np
import json
from glob import glob

In [6]:
# load the scalers
path_scalers = '../02_CreateRecords/PanStarrs/Folds/*/scalers/*.json'
all_scalers = glob(path_scalers)

# Filter by the relevant ones
relevant = ['T_eff','Radius']
scalers = {}
for rel in relevant:
    scalers[rel] = [i for i in all_scalers if rel in i]

In [12]:
# work on the T_eff one
param = 'T_eff'
teff_scal = scalers[param]

# Load the scalers
scal = []
for path in teff_scal:
    with open(path, 'r') as file:
        scal.append(json.load(file))


In [13]:
scal

[{'mean': 4413.430984359401, 'std': 2984.8569600738647},
 {'mean': 4391.681567431803, 'std': 2960.919261403178},
 {'mean': 4379.436123752495, 'std': 2984.0965141109505},
 {'mean': 4399.774530715475, 'std': 2975.457352811763},
 {'mean': 4412.90255906822, 'std': 3008.6029392316827},
 {'mean': 4374.435207886855, 'std': 2977.241728225244},
 {'mean': 4361.781684065203, 'std': 3014.995445295148}]

In [43]:
# Load folds with the raw data
path_folds_val = '../01_CreateFolds/PanStarrs/Folds/*/val.csv'
path_folds_train= '../01_CreateFolds/PanStarrs/Folds/*/train.csv'
path_folds_test= '../01_CreateFolds/PanStarrs/Folds/test.csv'
# Get all the files
folds_train = glob(path_folds_train)
folds_train.sort()
folds_val = glob(path_folds_val)
folds_val.sort()
# Get the train and val files open
dfs_train = [pd.read_csv(fold, usecols=['ID']+relevant) for fold in folds_train]
dfs_val = [pd.read_csv(fold, usecols=['ID']+relevant) for fold in folds_val]
df_test = pd.read_csv(path_folds_test)

In [44]:
# Load the folds with the data passed thorugh the pipeline
import sys
import os
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Avoids warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import tensorflow as tf
sys.path.append('../../../../../src/')
import ScalableLib.classifier.Multiband as multiband

survey = 'PanStarrs'
path = os.path.join('../02_CreateRecords/', survey, 'Folds/Fold_*',)
folds = glob(path)
folds.sort()

if not os.path.exists('./Results'):
    os.mkdir('./Results')

train_args = {
            'hidden_size_bands':[64, 64, 64],
            'hidden_size_central':[64, 64, 64],
            'fc_layers_bands':[128,128],
            'fc_layers_central':[128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':1000,
            'num_threads':7,
            'batch_size':512,
            'dropout':0.35,
            'lr':[[5e-3]*5, 2.5e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':500, 
            'use_class_weights':False,# Not Used
            'mode' : 'classifier+regression',
            }
# loss_weights = {'Class':300.0, 'T_eff':1.0,'Radius':1e0}
loss_weights = {'Class':300.0, 'T_eff':20.0,'Radius':1e0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': ['T_eff', 'Radius'],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,
                    'N_skip' : 2, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': False,
                    'train_steps_central' : 1,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args    
                    }



In [45]:
news = []
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'

    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../02_CreateRecords/'+survey+'/', './').replace('/Folds/', '/Results/')
    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)
        
    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = base_dir+'metadata_preprocess.json'
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers')
    # Define the train args
    train_args = {**train_args, **train_args_specific}


    train_files = base_dir+'train/*.tfrecord'
    val_files = base_dir+'val/*.tfrecord'
    test_files = base_dir+'test/*.tfrecord'

    new = multiband.Network()
    news.append(new)
    try:
        new.train(train_args, train_files, val_files, test_files)
    # new.train_loop()
    except:
            continue

./Results/Fold_1/Models/20241204-2014
./Results/Fold_2/Models/20241204-2014


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:1210: UserWarning: Layer 'rnn_layers_bands' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encoutered: ''missing a required argument: 'sequences'''
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:359: UserWarning: `build()` was called on layer 'rnn_layers_bands', however the layer does not have 

./Results/Fold_3/Models/20241204-2014
./Results/Fold_4/Models/20241204-2014
./Results/Fold_5/Models/20241204-2014
./Results/Fold_6/Models/20241204-2014
./Results/Fold_7/Models/20241204-2014


In [46]:
df_dataset = []
for new in news:
    df_temp = []
    for batch in new.dataset_test:
        ids = batch[0]['ID'].numpy().astype(np.int64)
        teff = batch[1]['T_eff'].numpy()
        radius = batch[1]['Radius'].numpy()
        df = pd.DataFrame(data = {'T_eff':teff, 'Radius':radius}, index=ids)
        df_temp.append(df)
    df_dataset.append(pd.concat(df_temp, axis=0))
        # df_teff = df[df.T_eff>0].drop('Radius', axis=1)
        # df_rad = df[df.Radius>0].drop('T_eff', axis=1)

2024-12-04 20:14:51.412846: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-04 20:14:51.548476: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-04 20:14:51.679598: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-04 20:14:51.811021: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-04 20:14:51.944391: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-04 20:14:52.075861: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-04 20:14:52.207950: W tensorflow/core/framework/local_rendezvous.cc:404] L

Now, let's evaluate the processed data from `df_dataset` with the raw data (`df_test`, `dfs_train`, `dfs_val`)processed with the scalers in `scal` for Teff.

In [88]:
# # Get R2 to see which is the correct scaler
# from sklearn.metrics import r2_score
# df_raw = df_test.copy()[['ID']+relevant].set_index('ID')

# # Compare the processed data, see if we have a problem
# raw_transform = ((df_raw[[param]]-sca['mean'])/sca['std']).rename({param:param+'_raw'}, axis=1)
# rnn_transform =df_rnn.copy().rename({param:param+'_rnn'}, axis=1)
# comparison_param = pd.merge(raw_transform, rnn_transform, left_index=True, right_index=True)

# # Select the first fold for the processed
# i =0
# param = 'T_eff'
# df_rnn = df_dataset[i]
# df_rnn = df_rnn[df_rnn[param]>0][[param]]

# # Select the test raw data and the first scaler
# sca = scal[i]


# r2 = r2_score(comparison_param[param+'_raw'], comparison_param[param+'_rnn'])


0.9984623759672677

In [92]:
# Get R2 to see which is the correct scaler
from sklearn.metrics import r2_score

param = 'T_eff'
df_raw = df_test.copy()[['ID']+relevant].set_index('ID')
r2s = {}
comparisons = {}
for i in range(len(scal)):
    # Select the folds
    # Select the test raw data and the first scaler
    sca = scal[i]

    # Compare the processed data, see if we have a problem
    raw_transform = ((df_raw[[param]]-sca['mean'])/sca['std']).rename({param:param+'_raw'}, axis=1)

    # Get the network data
    df_rnn = df_dataset[i]
    df_rnn = df_rnn[df_rnn[param]>0][[param]]
    rnn_transform =df_rnn.copy().rename({param:param+'_rnn'}, axis=1)
    
    comparison_param = pd.merge(raw_transform, rnn_transform, left_index=True, right_index=True)
    
    r2 = r2_score(comparison_param[param+'_raw'], comparison_param[param+'_rnn'])
    # Store the values
    r2s[i]=r2
    comparisons[i] = comparison_param

In [99]:
comparisons[2]

,T_eff_raw,T_eff_rnn
90752870729198448,0.554125,0.554117
111850650934828506,0.788032,0.787658
115542702349595209,0.539828,0.539842
74652707998828066,0.133228,0.133878
87362697938833009,0.772282,0.771932
...,...,...
93772523605923026,0.019347,0.020176
128132569795921034,1.161344,1.160386
98022186269379486,0.722903,0.722630
137421240249180848,1.055485,1.054692


In [93]:
r2s

{0: 0.9984623759672677,
 1: 0.9998866260041654,
 2: 0.9999969532960324,
 3: 0.9997286934594699,
 4: 0.9966397758292475,
 5: 0.9998399416038822,
 6: 0.999910620790257}

In [103]:
df_raw[df_raw.T_eff>0].mean()

T_eff     6222.250592
Radius       1.285679
dtype: float64

In [104]:
sca

{'mean': 4361.781684065203, 'std': 3014.995445295148}

In [119]:
# COmpute the scaler
i=0
df_train_i = dfs_train[i][dfs_train[i][param]>0].set_index('ID')[[param]]
# df_train_i = dfs_train[i].set_index('ID')[[param]]
mean_i = df_train_i.mean()
std_i = df_train_i.std()
mean_i, std_i

(T_eff    6215.638957
 dtype: float64,
 T_eff    1122.506397
 dtype: float64)

In [116]:
scal[0]

{'mean': 4413.430984359401, 'std': 2984.8569600738647}

The scaler was being computed with the negative values. 